In [1]:
import os 
os.chdir(os.path.dirname(os.getcwd()))

In [2]:

from credential_accessor import CredentialAccessor 
from dao.google_bigquery import GoogleBigQuery as gbq 

creds_accessor = CredentialAccessor(env = "prod", on_server= False)
myGBQ = gbq(attr = creds_accessor.get_attr())


from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import itertools
import random

In [4]:
from datetime import datetime
datetime.now()

datetime.datetime(2024, 2, 19, 16, 37, 23, 852756)

In [9]:
import numpy as np
a= np.array([21,21,3,12,31])
list_ = np.array([[1,2,3],[3,2,1]])
a[list_]

array([[21,  3, 12],
       [12,  3, 21]])

### Num. pareto per cluster

In [3]:
with open(os.path.join('queries','test_count.sql'),'r') as f:
    pareto_per_cluster = myGBQ.gbq_read(f.read())
pareto_per_cluster

,cluster,count_pareto_prd
0,Madura,31
1,GSM,99
2,JLB,149
3,PML,106
4,PSB,110
5,Bocimi,168
6,Kapursub,138
7,Bandung Raya,199
8,Ciayumajakuning,171
9,Priangan Timur 1,157


In [4]:
def get_pareto(myGBQ, cluster_name : str, N_prd = 150,*args, **kwargs) -> np.array :
    with open(os.path.join(os.getcwd(),'queries','list_pareto.sql'),'r') as f:
        q_ = f.read()
    q_ = q_.format(cluster_name = cluster_name,N_prd = N_prd)
    df = myGBQ.gbq_read(q_)
    return df.prd_id.to_numpy()
def get_non_pareto(myGBQ, cluster_name : str, DATETIME_TRX_END : str, DATETIME_TRX_START : str, N_prd = 150,*args, **kwargs) -> np.array :
    with open(os.path.join(os.getcwd(),'queries','list_non_pareto.sql'),'r') as f:
        q_ = f.read()
    q_ = q_.format(cluster_name = cluster_name,N_prd = N_prd, START_TRX_PERIOD = DATETIME_TRX_START, END_TRX_PERIOD = DATETIME_TRX_END)
    df = myGBQ.gbq_read(q_)
    return df.prd_id.to_numpy()
    

In [37]:
DATETIME_TRX_END = datetime.now().strftime("%Y-%m-%d")
DATETIME_TRX_START= (datetime.now()+relativedelta(months=-3)).strftime("%Y-%m-%d")

#Get pareto and non_pareto group products (for pareto groups > 150)
non_pareto = get_non_pareto(myGBQ,cluster_name = "Bocimi",DATETIME_TRX_END = DATETIME_TRX_END, DATETIME_TRX_START=DATETIME_TRX_START)
pareto = get_pareto(myGBQ,cluster_name = "Bocimi")

#Split them and create 

def create_group_products(pareto,non_pareto,n_groups=15):
    pareto_permute_set = np.split(np.random.permutation(pareto),15)
    non_pareto_permute_set = np.split(np.random.permutation(non_pareto),15)
    group_products = np.zeros([n_groups,int((len(pareto)+len(non_pareto))/n_groups)])
    for idx,i in enumerate(zip(pareto_permute_set,non_pareto_permute_set)):
        group_products[idx,:] = np.random.permutation(np.concatenate([i[0],i[1]]))
    return group_products.astype(int)

random_basic = np.arange(20).astype(int)

def shift_right_list(list_):
    return [list_[-1]] + list_[:-1]

def create_random_set(arr, min_appear=3,set_size=9, *args, **kwargs):
    # Oversampling the list
    len_arr = len(arr)
    arr_sample = np.array(list(arr)*min_appear)
    len_arr_sample = len(arr_sample)
    mod_resid = len_arr_sample%set_size
    
    # Random sorting the list
    arr_sample = list(np.random.permutation(np.concatenate([arr_sample,np.random.choice(arr,set_size-mod_resid)])))
    print(f"Unordered pool :{arr_sample}")
    arranged_sample = np.zeros([int(len(arr_sample)/set_size),set_size])
    
    #Sort the list so that no set has two or more identical items.
    max_trials = set_size #Maximum trials for shifting 
    
    for i in range(arranged_sample.shape[0]):
        print(f"Creating product batch {i+1} ...")
        observed_list = arr_sample[set_size*i:(set_size*(i+1))] #create the chunk
        print(f"Unordered product batch {i+1} : {observed_list}")
        
        if i < arranged_sample.shape[0]-1 :
            for j in range(len(observed_list)):
                if len(observed_list[:j+1])==len(set(observed_list[:j+1])) : #No two or more identical items
                    print(f"Ordering product batch {i+1} : {observed_list[:j+1]}")
                else : # Identical items are found, shifting pool and the chunk
                    print(f"{observed_list[j]} already existed, replace this item by shifting the pool!")
                    trials = 0
                    while trials <= max_trials :
                        trials = trials + 1
                        print(f'Conducting trial {trials}...')
                        arr_sample = arr_sample[:(set_size*i+j)] + shift_right_list(arr_sample[(set_size*i+j):]) #Last element of the pool become the very first element in the observation
                        print(f"Pool has been ordered : {arr_sample}")
                        observed_list[j:] = arr_sample[(set_size*i+j):(set_size*(i+1))]
                        
                        if len(observed_list[:j+1])==len(set(observed_list[:j+1])) : 
                            print(f"Ordering product batch {i+1} : {observed_list[:j+1]}")
                            break 
                        else  : 
                            pass
        else : 
            print(f"Creating last product batch...")
            # Check if the sorting is possible 
            observed_list_arr = np.array(observed_list)
            bincount_last_batch = np.bincount(observed_list_arr)
            problematic_items = np.where(bincount_last_batch == 2)[0]
            for items_ in problematic_items:
                print(f"{items_} is found having duplicate, fixing this issue...") #Do resampling if an issue is found
                location_items = np.where(observed_list_arr == items_)[0]
                observed_list_arr[location_items[0]] = np.random.choice(arr[~np.isin(arr,np.array(list(set(observed_list_arr))))])
                print(f"Ordering product batch {i+1} : {observed_list_arr}")
            print(f"Ordering product batch {i+1} : {observed_list_arr}")
            observed_list = list(observed_list_arr)
            arr_sample[set_size*i:(set_size*(i+1))] = observed_list
                
                
            
        arranged_sample[i,:] = np.array(observed_list)
                    
    return arranged_sample.astype(int), np.bincount(np.array(arr_sample))

def check_row_similarity(arr_ : np.array, *args, **kwargs) :
    n_row = arr_.shape[0]
    pool_ = np.arange(n_row)
    val_ = False
    for i in itertools.combinations(pool_,2):
        if set(arr_[i[0],:]) == set(arr_[i[1],:]):
            val_ = True 
            break
    return val_
        
    



In [60]:
True or False

True

In [48]:
random_batch,bincount=create_random_set(random_basic)

Unordered pool :[14, 9, 2, 4, 12, 4, 1, 6, 19, 6, 3, 3, 12, 5, 13, 13, 16, 9, 11, 4, 8, 2, 16, 17, 5, 11, 13, 10, 16, 14, 1, 8, 6, 17, 10, 1, 19, 18, 11, 2, 15, 8, 15, 0, 17, 12, 19, 3, 7, 9, 0, 18, 16, 7, 15, 14, 18, 11, 10, 5, 7, 0, 5]
Creating product batch 1 ...
Unordered product batch 1 : [14, 9, 2, 4, 12, 4, 1, 6, 19]
Ordering product batch 1 : [14]
Ordering product batch 1 : [14, 9]
Ordering product batch 1 : [14, 9, 2]
Ordering product batch 1 : [14, 9, 2, 4]
Ordering product batch 1 : [14, 9, 2, 4, 12]
4 already existed, replace this item by shifting the pool!
Conducting trial 1...
Pool has been ordered : [14, 9, 2, 4, 12, 5, 4, 1, 6, 19, 6, 3, 3, 12, 5, 13, 13, 16, 9, 11, 4, 8, 2, 16, 17, 5, 11, 13, 10, 16, 14, 1, 8, 6, 17, 10, 1, 19, 18, 11, 2, 15, 8, 15, 0, 17, 12, 19, 3, 7, 9, 0, 18, 16, 7, 15, 14, 18, 11, 10, 5, 7, 0]
Ordering product batch 1 : [14, 9, 2, 4, 12, 5]
4 already existed, replace this item by shifting the pool!
Conducting trial 1...
Pool has been ordered : [14

In [49]:
check_row_similarity(random_batch)

False

In [58]:
bincount

array([3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3])

In [51]:
random_batch

array([[14,  9,  2,  4, 12,  5,  0,  7, 10],
       [ 5,  4,  1,  6, 19, 11, 18, 14, 15],
       [ 6,  3,  7, 16, 18,  0,  9, 19, 12],
       [ 3,  7, 17,  0, 15,  8,  2, 11, 18],
       [15,  3, 12,  5, 13, 19,  1, 10, 17],
       [13, 16,  9, 11,  4,  8,  2,  6,  1],
       [ 8,  1, 17,  5, 11, 13, 10, 16, 14]])

In [56]:
any(np.where(bincount < 2)[0])

False

In [57]:
any(bincount)

True

In [134]:
import itertools
import random
example = np.arange(20).astype(int)
def get_random_combi(arr, n_combi, r , *args, **kwargs):
    iterable_combi = itertools.combinations(arr,r)
    pool = tuple(iterable_combi)
    n_pool = len(pool)
    indices_ = sorted(random.sample(range(n_pool),n_combi))
    return tuple(pool[i] for i in indices_)

In [155]:
pareto_permute_set

[array([1929, 4874, 1022, 4314, 912, 136, 4315, 4036, 4226, 272],
       dtype=object),
 array([614, 468, 40, 7762, 8447, 361, 2817, 1375, 2073, 4646],
       dtype=object),
 array([8275, 6095, 86, 115, 12686, 938, 261, 16341, 233, 1609],
       dtype=object),
 array([130, 8554, 3330, 14035, 1612, 1161, 68, 173, 6054, 11117],
       dtype=object),
 array([171, 5567, 2349, 239, 3366, 33, 4671, 2462, 222, 4719],
       dtype=object),
 array([1446, 22137, 4553, 167, 613, 1325, 1154, 2438, 4162, 2241],
       dtype=object),
 array([56, 2245, 31, 22284, 2204, 4328, 2733, 142, 9982, 1279],
       dtype=object),
 array([226, 20, 6497, 725, 5788, 323, 5505, 1374, 998, 4322], dtype=object),
 array([997, 506, 7344, 5002, 232, 2022, 6503, 2268, 10704, 3407],
       dtype=object),
 array([8527, 5442, 135, 4131, 3451, 2430, 4333, 762, 3660, 144],
       dtype=object),
 array([816, 85, 13579, 7888, 73, 58, 3576, 398, 616, 3311], dtype=object),
 array([939, 4023, 35, 22340, 6228, 1457, 4334, 290, 24,

In [143]:
s = np.random.permutation(np.array(list(example)*3))
s

array([18, 12, 15,  1,  0, 19,  8, 14,  1, 19, 11,  6,  7,  7, 15, 12,  4,
        2,  2,  7,  5,  6,  5,  2,  3,  0, 18, 10,  3,  9, 19, 12,  9, 13,
        8,  6, 18, 14, 16, 15, 16, 17, 16,  0, 11, 17, 10,  3, 11, 17,  8,
        1, 13,  4, 14, 13,  5,  4, 10,  9])

In [ ]:
idx = 0
max_trial = 10
for i in len(s):
    if i==0 or i%9==8
        a = []
    else:
        pass
    a.append(s[i])
    if len(set(a)) == len(a) :
        pass
    else :
        trial = 0
        while trial < max_trial :
            s[i:] = s[
        
        
        
        

In [296]:
np.array([list([1,2,3])])

array([[1, 2, 3]])

In [149]:
True or False

True

In [153]:
17%9

8

In [297]:
np.random.choice([1,2,3])

1

In [11]:
for i in itertools.combinations([0,1,2,3,4,5,6],2):
    print(i[0])

0
0
0
0
0
0
1
1
1
1
1
2
2
2
2
3
3
3
4
4
5


In [35]:
np.array(list(set([2,3,4,5,5,6,3])))

array([2, 3, 4, 5, 6])

In [1]:
np.bincount([1,3,4])

NameError: name 'np' is not defined

In [2]:
np.array(1)

NameError: name 'np' is not defined